<br>

# Introdução

Por meio do [_site_ da Defesa Civil](http://www.defesacivil.sp.gov.br/) (mais específicamente nesse [_link_](http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/)) foi possível obter diversas informações sobre os Coordenadores que atuam nas 'Coordenadorias Regionais de Proteção e Defesa Civil - REPDEC'.


In [1]:
import os
import random
import re
import time

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from osgeo import gdal, osr
from tqdm.notebook import tqdm, trange

from my_paths import output_path_tabs

<br>

# Dados Tabulares


## Scrapy _Site_

Iniciamente foi obtido o código fonte da paǵina, usando o _BeautifulSoup_.


In [2]:
r = requests.get('http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/')
soup = BeautifulSoup(r.content, 'html.parser')
content_tag = soup.find('div', {'id': 'content'})


In [4]:
list_text = []
for i in content_tag.find_all('p'):
    print(i.text)
    list_text.append(i.text.replace('\xa0', ' '))


list_text

REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
COORDENADOR REGIONAL: ALFREDO PISANI
E-mail: apisani@sp.gov.br
Telefone: (11) 3293-8585
 
COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM CESAR TADEU RIBEIRO FLORINDO
E-mail: cesartadeu@policiamilitar.sp.gov.br
Telefone: (11)  3396-2604
 
COORDENADOR REGIONAL ADJUNTO 2: EDSON BISPO DOS SANTOS
E-mail: edsonbispo@prefeitura.sp.gov.br
Telefone: (11)  3124-5157 ramal 178
 
COORDENADORA REGIONAL ADJUNTO 3: LUIZA PACHECO DA SILVA
E-mail: luizapacheco@prefeitura.sp.gov.br
Telefone: (11)  3298-7045
 
REGIÃO METROPOLITANA DE SANTO ANDRÉ – REPDEC / M-2
COORDENADOR REGIONAL: CAPITÃO PM ALEXANDRE ALVES MOTA
E-mail: alexandremota@policiamilitar.sp.gov.br
Telefone: (11) 4125-9108
 
COORDENADOR REGIONAL ADJUNTO 1: 1º TENENTE PM NELIO DANIEL PERANTONI
E-mail: nelioperantoni@policiamilitar.sp.gov.br
Telefone: (11) 4056-6274
 
COORDENADOR REGIONAL ADJUNTO 2: 1º TENENTE PM LUÍS FERNANDO VIEIRA
E-mail: luisfv@policiamilitar.sp.gov.br
Telefone: (11) 4056-6274
 
C

['REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1',
 'COORDENADOR REGIONAL: ALFREDO PISANI',
 'E-mail: apisani@sp.gov.br',
 'Telefone: (11) 3293-8585',
 ' ',
 'COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM CESAR TADEU RIBEIRO FLORINDO',
 'E-mail: cesartadeu@policiamilitar.sp.gov.br',
 'Telefone: (11)  3396-2604',
 ' ',
 'COORDENADOR REGIONAL ADJUNTO 2: EDSON BISPO DOS SANTOS',
 'E-mail: edsonbispo@prefeitura.sp.gov.br',
 'Telefone: (11)  3124-5157 ramal 178',
 ' ',
 'COORDENADORA REGIONAL ADJUNTO 3: LUIZA PACHECO DA SILVA',
 'E-mail: luizapacheco@prefeitura.sp.gov.br',
 'Telefone: (11)  3298-7045',
 ' ',
 'REGIÃO METROPOLITANA DE SANTO ANDRÉ – REPDEC / M-2',
 'COORDENADOR REGIONAL: CAPITÃO PM ALEXANDRE ALVES MOTA',
 'E-mail: alexandremota@policiamilitar.sp.gov.br',
 'Telefone: (11) 4125-9108',
 ' ',
 'COORDENADOR REGIONAL ADJUNTO 1: 1º TENENTE PM NELIO DANIEL PERANTONI',
 'E-mail: nelioperantoni@policiamilitar.sp.gov.br',
 'Telefone: (11) 4056-6274',
 ' ',
 'COORDENADOR REGIONAL ADJUNTO 

<br>

## Ajuste da tabela

A partir disso, montei uma lista com todos os valores que estavam na página e converti a lista em uma tabela. Adicionei uma coluna contendo o valor do índice quando, na linha da tabela, houvesse a palavra "REPDEC e preenchi essa tabela.


In [5]:
# Create Table
df = pd.DataFrame({'data': list_text})
df.head()

,data
0,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
1,COORDENADOR REGIONAL: ALFREDO PISANI
2,E-mail: apisani@sp.gov.br
3,Telefone: (11) 3293-8585
4,


In [6]:
df = df[df['data'] != ' ']
df

,data
0,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
1,COORDENADOR REGIONAL: ALFREDO PISANI
2,E-mail: apisani@sp.gov.br
3,Telefone: (11) 3293-8585
5,COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM CES...
...,...
422,E-mail: freitasthiago@policiamilitar.sp.gov.br
423,Telefone: (15) 3531-1711
425,COORDENADOR REGIONAL ADJUNTO 5: 2° SARGENTO PM...
426,E-mail: rafaeljulio@policiamilitar.sp.gov.br


In [7]:
# Define os marcos iniciais
df['repdec'] = df.apply(
    lambda x: x['data'] if 'REPDEC' in x['data'] else np.nan, axis=1
)

# Result
df.head(10)

,data,repdec
0,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
1,COORDENADOR REGIONAL: ALFREDO PISANI,NaN
2,E-mail: apisani@sp.gov.br,NaN
3,Telefone: (11) 3293-8585,NaN
5,COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM CES...,NaN
6,E-mail: cesartadeu@policiamilitar.sp.gov.br,NaN
7,Telefone: (11) 3396-2604,NaN
9,COORDENADOR REGIONAL ADJUNTO 2: EDSON BISPO DO...,NaN
10,E-mail: edsonbispo@prefeitura.sp.gov.br,NaN
11,Telefone: (11) 3124-5157 ramal 178,NaN


In [8]:
# Preenche Coluna
df['repdec'] = df['repdec'].ffill()

# Result
df.head(10)

,data,repdec
0,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
1,COORDENADOR REGIONAL: ALFREDO PISANI,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
2,E-mail: apisani@sp.gov.br,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
3,Telefone: (11) 3293-8585,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
5,COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM CES...,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
6,E-mail: cesartadeu@policiamilitar.sp.gov.br,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
7,Telefone: (11) 3396-2604,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
9,COORDENADOR REGIONAL ADJUNTO 2: EDSON BISPO DO...,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
10,E-mail: edsonbispo@prefeitura.sp.gov.br,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
11,Telefone: (11) 3124-5157 ramal 178,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1


In [9]:
# dddd
#df = df[df['data'] != df['repdec']]

# Elimina Linhas com NaN
df = df.dropna()

# Result
df.head(10)

,data,repdec
0,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
1,COORDENADOR REGIONAL: ALFREDO PISANI,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
2,E-mail: apisani@sp.gov.br,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
3,Telefone: (11) 3293-8585,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
5,COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM CES...,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
6,E-mail: cesartadeu@policiamilitar.sp.gov.br,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
7,Telefone: (11) 3396-2604,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
9,COORDENADOR REGIONAL ADJUNTO 2: EDSON BISPO DO...,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
10,E-mail: edsonbispo@prefeitura.sp.gov.br,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1
11,Telefone: (11) 3124-5157 ramal 178,REGIÃO METROPOLITANA DE SÃO PAULO – REPDEC / M-1


<br>

Criei sub-listas, de tamanhos diferentes, para cada uma desses conjuntos, por meio desse índice e converti em uma tabela.


In [10]:
list_temps = []
for i in list(set(df['repdec'])):
    df_temp = df[df['repdec'] == i]
    list_temp = list(df_temp['data'])
    list_temps.append(list_temp)


In [11]:
# 
n_columns = max([len(x) for x in list_temps])
n_columns_exclude_repdec = n_columns -1

# 
if n_columns_exclude_repdec % 3 == 0:
    n_coords = int(n_columns_exclude_repdec / 3)

# Results
n_coords

9

In [12]:
# Cria tabela
df = pd.DataFrame(
    list_temps,
)

# Results
print(len(df.columns))
df.head()

28


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,REGIÃO METROPOLITANA DE SANTO ANDRÉ – REPDEC /...,COORDENADOR REGIONAL: CAPITÃO PM ALEXANDRE ALV...,E-mail: alexandremota@policiamilitar.sp.gov.br,Telefone: (11) 4125-9108,COORDENADOR REGIONAL ADJUNTO 1: 1º TENENTE PM ...,E-mail: nelioperantoni@policiamilitar.sp.gov.br,Telefone: (11) 4056-6274,COORDENADOR REGIONAL ADJUNTO 2: 1º TENENTE PM ...,E-mail: luisfv@policiamilitar.sp.gov.br,Telefone: (11) 4056-6274,...,Telefone: (11) 96416-5741,COORDENADOR REGIONAL ADJUNTO 6: 1º TENENTE PM ...,E-mail: lbariani@policiamilitar.sp.gov.br,Telefone: (11) 99595-6420,None,None,None,None,None,None
1,REGIÃO METROPOLITANA DE OSASCO – REPDEC / M-4,COORDENADOR REGIONAL: CAPITÃO PM THIAGO OLIVEI...,E-mail: tbelucci@policiamilitar.sp.gov.br,Telefone: (11) 4199-2955,COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM DAN...,E-mail: dboros@policiamilitar.sp.gov.br,Telefone:(11) 4616-2933,COORDENADOR REGIONAL ADJUNTO 2: 1º TENENTE PM ...,E-mail: debarros@policiamilitar.sp.gov.br,Telefone: (11) 4704-5572,...,Telefone: (11) 4787-5775,None,None,None,None,None,None,None,None,None
2,REGIÃO DE REGISTRO – REPDEC / I-1,COORDENADORA REGIONAL: KATHLEEN GOMES DA SILVA...,E-mail: kathleen.chaves6@gmail.com,Telefone: (13) 3821-3244,COORDENADOR REGIONAL ADJUNTO 1: CÉLIO DE FÁRIAS,E-mail: celio.de.farias@hotmail.com,Telefone: (13) 3845-4806,COORDENADORA REGIONAL ADJUNTO 2: VANESSA VICEN...,E-mail: vanessavicentini0502@gmail.com,Telefone: (13) 99633-2996,...,None,None,None,None,None,None,None,None,None,None
3,REGIÃO DE SÃO JOSÉ DO RIO PRETO – REPDEC / I-8,COORDENADOR REGIONAL: CAPITÃO PM ALEXANDRE OBV...,E-mail: aobvioslo@policiamilitar.sp.gov.br,Telefone:(17) 3211-8870,COORDENADOR REGIONAL ADJUNTO 1: 1º TENENTE PM ...,E-mail: carbonera@policiamilitar.sp.gov.br,Telefone: (17) 3211-8870,COORDENADOR REGIONAL ADJUNTO 2: 1º SARGENTO PM...,E-mail: rjnones@policiamilitar.sp.gov.br,Telefone: (11) 94149-3024,...,Telefone: (17) 3211-8870,COORDENADOR REGIONAL ADJUNTO 6: 1º SARGENTO PM...,E-mail: rodrigobiazi@policiamilitar.sp.gov.br,Telefone: (17) 3211-8870,COORDENADOR REGIONAL ADJUNTO 7: 2º SARGENTO PM...,E-mail: bindela@policiamilitar.sp.gov.br,Telefone: (17) 3211-8870,None,None,None
4,REGIÃO DE BAURU – REPDEC / I-7,COORDENADOR REGIONAL: TENENTE CORONEL PM HELDE...,E-mail: helderkato@policiamilitar.sp.gov.br,Telefone: (14) 3222-7474,COORDENADOR REGIONAL ADJUNTO 1: CAPITÃO PM VIC...,E-mail: tozi@policiamilitar.sp.gov.br,Telefone: (14) 3222-7474,COORDENADOR REGIONAL ADJUNTO 2: 1º TENENTE PM ...,E-mail: viniciusburin@policiamilitar.sp.gov.br,Telefone: (14) 3231-2359,...,Telefone: (14) 3662-4911,COORDENADOR REGIONAL ADJUNTO 6: 1º SARGENTO TA...,E-mail: tadeu.bergamasco10@gmail.com,Telefone: (14) 3662-4911,COORDENADOR REGIONAL ADJUNTO 7: EVANDRO ANTONI...,E-mail: defesacivil@cabralia.sp.gov.br,Telefone: (14) 3285-1145,COORDENADOR REGIONAL ADJUNTO 8: MARCELO RYAL DIAS,E-mail: marcelodias@bauru.sp.gov.br,Telefone: (14) 3235-1169


In [13]:
list_cols = []
list_itens = ['nome', 'mail', 'tel']

# Adiciona Primeira Coluna
list_cols.extend(['repdec'])

# Adiciona Colunas Seguintes (coord_regional)
list_cols.extend([f'coord_regional_{x}' for x in list_itens])

# Adiciona Primeira Coluna (coord_adj)
for n_coord_adj in range(8):    
    n_coord_adj = n_coord_adj+1
    list_cols.extend([f'coord_adj{n_coord_adj}_{x}' for x in list_itens])


# Results
print(len(list_cols))
list_cols


28


['repdec',
 'coord_regional_nome',
 'coord_regional_mail',
 'coord_regional_tel',
 'coord_adj1_nome',
 'coord_adj1_mail',
 'coord_adj1_tel',
 'coord_adj2_nome',
 'coord_adj2_mail',
 'coord_adj2_tel',
 'coord_adj3_nome',
 'coord_adj3_mail',
 'coord_adj3_tel',
 'coord_adj4_nome',
 'coord_adj4_mail',
 'coord_adj4_tel',
 'coord_adj5_nome',
 'coord_adj5_mail',
 'coord_adj5_tel',
 'coord_adj6_nome',
 'coord_adj6_mail',
 'coord_adj6_tel',
 'coord_adj7_nome',
 'coord_adj7_mail',
 'coord_adj7_tel',
 'coord_adj8_nome',
 'coord_adj8_mail',
 'coord_adj8_tel']

In [14]:
# Cria tabela
df = pd.DataFrame(
    list_temps,
    columns=list_cols
)

<br>

Com a tabela bruta, passei a limpa-la, "splitando" os campos pelo caractere ":", além de outras correções.


In [15]:
# Exceção: em uma célula há duas vezes o ":"
df['coord_adj1_tel'] = (
    df['coord_adj1_tel']
    .astype(str)
    .apply(lambda x: x.replace('Ramal:', 'Ramal '))
)

In [16]:
# Remove tudo que vem antes do :
df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')

C:\Users\michel\AppData\Local\Temp\ipykernel_528\2708554829.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')


In [17]:
# Divide Células em Duas Colunas
df[['repdec_nome', 'repdec_cod']] = df['repdec'].str.split('–', expand=True)
df[['del', 'repdec_cod']] = df['repdec'].str.split('/', expand=True)
df.drop(['del', 'repdec'], axis=1, inplace=True, errors='ignore')

# Results
df.head()

,coord_regional_nome,coord_regional_mail,coord_regional_tel,coord_adj1_nome,coord_adj1_mail,coord_adj1_tel,coord_adj2_nome,coord_adj2_mail,coord_adj2_tel,coord_adj3_nome,...,coord_adj6_mail,coord_adj6_tel,coord_adj7_nome,coord_adj7_mail,coord_adj7_tel,coord_adj8_nome,coord_adj8_mail,coord_adj8_tel,repdec_nome,repdec_cod
0,CAPITÃO PM ALEXANDRE ALVES MOTA,alexandremota@policiamilitar.sp.gov.br,(11) 4125-9108,1º TENENTE PM NELIO DANIEL PERANTONI,nelioperantoni@policiamilitar.sp.gov.br,(11) 4056-6274,1º TENENTE PM LUÍS FERNANDO VIEIRA,luisfv@policiamilitar.sp.gov.br,(11) 4056-6274,1º TENENTE PM THIAGO CORDEIRO DE LIMA,...,lbariani@policiamilitar.sp.gov.br,(11) 99595-6420,None,None,None,None,None,None,REGIÃO METROPOLITANA DE SANTO ANDRÉ,M-2
1,CAPITÃO PM THIAGO OLIVEIRA BELUCCI,tbelucci@policiamilitar.sp.gov.br,(11) 4199-2955,CAPITÃO PM DANIEL BOROS MOTTA DE OLIVEIRA,dboros@policiamilitar.sp.gov.br,(11) 4616-2933,1º TENENTE PM LEANDRO LICO NEVES DE BARROS,debarros@policiamilitar.sp.gov.br,(11) 4704-5572,1º TENENTE PM ULISSES LEVI DUARTE DA SILVA,...,None,None,None,None,None,None,None,None,REGIÃO METROPOLITANA DE OSASCO,M-4
2,KATHLEEN GOMES DA SILVA CHAVES,kathleen.chaves6@gmail.com,(13) 3821-3244,CÉLIO DE FÁRIAS,celio.de.farias@hotmail.com,(13) 3845-4806,VANESSA VICENTINI DE ALMEIDA SANTOS,vanessavicentini0502@gmail.com,(13) 99633-2996,CAPITÃO PM LUÍS GABRIEL MATHEUS DE SOUSA,...,None,None,None,None,None,None,None,None,REGIÃO DE REGISTRO,I-1
3,CAPITÃO PM ALEXANDRE OBVIOSLO NETO,aobvioslo@policiamilitar.sp.gov.br,(17) 3211-8870,1º TENENTE PM MAIARA PRADO CARBONERA,carbonera@policiamilitar.sp.gov.br,(17) 3211-8870,1º SARGENTO PM ROBSON DE JESUS NONES,rjnones@policiamilitar.sp.gov.br,(11) 94149-3024,SUBTENENTE PM BRENO VIANA DE PAULA,...,rodrigobiazi@policiamilitar.sp.gov.br,(17) 3211-8870,2º SARGENTO PM CÁSSIO BINDELA MARTINS,bindela@policiamilitar.sp.gov.br,(17) 3211-8870,None,None,None,REGIÃO DE SÃO JOSÉ DO RIO PRETO,I-8
4,TENENTE CORONEL PM HELDER HIDEAKI KATO,helderkato@policiamilitar.sp.gov.br,(14) 3222-7474,CAPITÃO PM VICTOR FELIX TOZI BOMFIM,tozi@policiamilitar.sp.gov.br,(14) 3222-7474,1º TENENTE PM VINICIUS ALEXANDRE BURIN,viniciusburin@policiamilitar.sp.gov.br,(14) 3231-2359,2º TENENTE PM RES WAGNER OLIVEIRA SAONCELLA,...,tadeu.bergamasco10@gmail.com,(14) 3662-4911,EVANDRO ANTONIO CAVARSAN,defesacivil@cabralia.sp.gov.br,(14) 3285-1145,MARCELO RYAL DIAS,marcelodias@bauru.sp.gov.br,(14) 3235-1169,REGIÃO DE BAURU,I-7


## Funções

Funções para renomear campos


In [18]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  ': ' ',
        # Basics
        ' Com ': ' com ',
        ' Sobre ': ' sobre ',
        ' Da ': ' da ',
        ' De ': ' de ',
        ' Do ': ' do ',
        ' Das ': ' das ',
        ' Dos ': ' dos ',
        ' A ': ' a ',
        ' As ': ' as ',
        ' Ao ': ' ao ',
        ' Aos ': ' aos ',
        ' E ': ' e ',
        ' O ': ' o ',
        ' Os ': ' os ',
        # Erros
        '1ºten': '1º Ten',
        # Abreviações
        'Subten ': 'Subtenente ',
        'Sub Ten': 'Subtenente ',
        'Cap ': 'Capitão ',
        'Ten ': 'Tenente ',
        'Maj ': 'Major ',
        'Cel ': 'Coronel ',
        'Sgt ': 'Sargento ',
        ' Pm ': ' PM ',
        # Empty
        'None': '',
        'none': '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [19]:
def rename_tel(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  ': ' ',
        # Basics
        ' – ': '-',
        ')': ') ',
        ' (': ' / (',
        ' / ': '/',
        '//': '/',
        '/': ' / ',
        '  ': ' ',
        # Empty
        'None': '',
        'none': '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [20]:
def rename_mail(x):
    x = x.lower()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  ': ' ',
        # Basic
        ' ': '/',
        ';': '/',
        ',': '/',
        '//': '/',
        '/': ' / ',
        '  ': ' ',
        # Empty
        'None': '',
        'none': '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

<br>

## Renomeando Campos em Colunas


In [21]:
# Definindo lista de colunas que serão empregadas funções.
list_nome = []
list_tel = []
list_mail = []

for col in list(df.columns):
    if 'nome' in col:
        list_nome.append(col)
    if 'tel' in col:
        list_tel.append(col)
    if 'mail' in col:
        list_mail.append(col)

# Adicionando manualmente outras colunas.
list_nome.append('repdec_nome')
list_nome.append('repdec_cod')

# Aplicando funções
for field in list_nome:
    df[field] = df[field].astype(str).apply(lambda x: rename_nome(x))
for field in list_tel:
    df[field] = df[field].astype(str).apply(lambda x: rename_tel(x))
for field in list_mail:
    df[field] = df[field].astype(str).apply(lambda x: rename_mail(x))

<br>

E, por fim, reordenando colunas.


In [22]:
# Reordena Colunas
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df.reindex(cols, axis=1)

# Resultados
df.head()

,repdec_nome,repdec_cod,coord_regional_nome,coord_regional_mail,coord_regional_tel,coord_adj1_nome,coord_adj1_mail,coord_adj1_tel,coord_adj2_nome,coord_adj2_mail,...,coord_adj5_tel,coord_adj6_nome,coord_adj6_mail,coord_adj6_tel,coord_adj7_nome,coord_adj7_mail,coord_adj7_tel,coord_adj8_nome,coord_adj8_mail,coord_adj8_tel
0,Região Metropolitana de Santo André,M-2,Capitão PM Alexandre Alves Mota,alexandremota@policiamilitar.sp.gov.br,(11) 4125-9108,1º Tenente PM Nelio Daniel Perantoni,nelioperantoni@policiamilitar.sp.gov.br,(11) 4056-6274,1º Tenente PM Luís Fernando Vieira,luisfv@policiamilitar.sp.gov.br,...,(11) 96416-5741,1º Tenente PM Lorenzo Guimarães Bariani,lbariani@policiamilitar.sp.gov.br,(11) 99595-6420,,,,,,
1,Região Metropolitana de Osasco,M-4,Capitão PM Thiago Oliveira Belucci,tbelucci@policiamilitar.sp.gov.br,(11) 4199-2955,Capitão PM Daniel Boros Motta de Oliveira,dboros@policiamilitar.sp.gov.br,(11) 4616-2933,1º Tenente PM Leandro Lico Neves de Barros,debarros@policiamilitar.sp.gov.br,...,(11) 4787-5775,,,,,,,,,
2,Região de Registro,I-1,Kathleen Gomes da Silva Chaves,kathleen.chaves6@gmail.com,(13) 3821-3244,Célio de Fárias,celio.de.farias@hotmail.com,(13) 3845-4806,Vanessa Vicentini de Almeida Santos,vanessavicentini0502@gmail.com,...,,,,,,,,,,
3,Região de São José do Rio Preto,I-8,Capitão PM Alexandre Obvioslo Neto,aobvioslo@policiamilitar.sp.gov.br,(17) 3211-8870,1º Tenente PM Maiara Prado Carbonera,carbonera@policiamilitar.sp.gov.br,(17) 3211-8870,1º Sargento PM Robson de Jesus s,rjs@policiamilitar.sp.gov.br,...,(17) 3211-8870,1º Sargento PM Rodrigo de Biazi Xavier Silva,rodrigobiazi@policiamilitar.sp.gov.br,(17) 3211-8870,2º Sargento PM Cássio Bindela Martins,bindela@policiamilitar.sp.gov.br,(17) 3211-8870,,,
4,Região de Bauru,I-7,Tenente Coronel PM Helder Hideaki Kato,helderkato@policiamilitar.sp.gov.br,(14) 3222-7474,Capitão PM Victor Felix Tozi Bomfim,tozi@policiamilitar.sp.gov.br,(14) 3222-7474,1º Tenente PM Vinicius Alexandre Burin,viniciusburin@policiamilitar.sp.gov.br,...,(14) 3662-4911,1º Sargento Tadeu Bergamasco Urrea,tadeu.bergamasco10@gmail.com,(14) 3662-4911,Evandro Antonio Cavarsan,defesacivil@cabralia.sp.gov.br,(14) 3285-1145,Marcelo Ryal Dias,marcelodias@bauru.sp.gov.br,(14) 3235-1169


## _Splita_ Telefones


In [23]:
# Definindo lista de colunas que serão empregadas funções.
list_tel = []

for col in list(df.columns):
    if 'tel' in col:
        list_tel.append(col)

# Aplicando funções
for field in list_tel:
    tel_01 = f'{field}01'
    tel_02 = f'{field}02'
    tel_01ramal = f'{field}01ramal'
    tel_02ramal = f'{field}02ramal'

    # Split Telefone
    try:
        df[tel_01] = df[field].str.split(' / ', n=1, expand=True)[0]
    except:
        df[tel_01] = None
    try:
        df[tel_01ramal] = df[tel_01].str.split(' Ramal ', n=1, expand=True)[1]
    except:
        df[tel_01ramal] = None
    try:
        df[tel_01] = df[tel_01].str.split(' Ramal ', n=1, expand=True)[0]
    except:
        df[tel_01] = None

    try:
        df[tel_02] = df[field].str.split(' / ', n=1, expand=True)[1]
    except:
        df[tel_02] = None
    try:
        df[tel_02ramal] = df[tel_02].str.split(' Ramal ', n=1, expand=True)[1]
    except:
        df[tel_02ramal] = None
    try:
        df[tel_02] = df[tel_02].str.split(' Ramal ', n=1, expand=True)[0]
    except:
        df[tel_02] = None

    # Deleta Coluna
    df.drop(field, axis=1, inplace=True, errors='ignore')

## _Splita_ E-mails


In [24]:
list_mail = []

for col in list(df.columns):
    if 'mail' in col:
        list_mail.append(col)

# Aplicando funções
for field in list_mail:
    mail_01 = f'{field}01'
    mail_02 = f'{field}02'

    # Split Telefone
    try:
        df[mail_01] = df[field].str.split(' / ', n=1, expand=True)[0]
    except:
        df[mail_01] = None
    try:
        df[mail_02] = df[field].str.split(' / ', n=1, expand=True)[1]
    except:
        df[mail_02] = None

    # Deleta Coluna
    df.drop(field, axis=1, inplace=True, errors='ignore')

## Reordena Colunas


In [25]:
# Reordena Colunas
cols = df.columns.tolist()

cols.sort()
cols = cols[-2:] + cols[-9:-2] + cols[:-9]

df = df.reindex(cols, axis=1)
cols

['repdec_cod',
 'repdec_nome',
 'coord_regional_mail01',
 'coord_regional_mail02',
 'coord_regional_nome',
 'coord_regional_tel01',
 'coord_regional_tel01ramal',
 'coord_regional_tel02',
 'coord_regional_tel02ramal',
 'coord_adj1_mail01',
 'coord_adj1_mail02',
 'coord_adj1_nome',
 'coord_adj1_tel01',
 'coord_adj1_tel01ramal',
 'coord_adj1_tel02',
 'coord_adj1_tel02ramal',
 'coord_adj2_mail01',
 'coord_adj2_mail02',
 'coord_adj2_nome',
 'coord_adj2_tel01',
 'coord_adj2_tel01ramal',
 'coord_adj2_tel02',
 'coord_adj2_tel02ramal',
 'coord_adj3_mail01',
 'coord_adj3_mail02',
 'coord_adj3_nome',
 'coord_adj3_tel01',
 'coord_adj3_tel01ramal',
 'coord_adj3_tel02',
 'coord_adj3_tel02ramal',
 'coord_adj4_mail01',
 'coord_adj4_mail02',
 'coord_adj4_nome',
 'coord_adj4_tel01',
 'coord_adj4_tel01ramal',
 'coord_adj4_tel02',
 'coord_adj4_tel02ramal',
 'coord_adj5_mail01',
 'coord_adj5_mail02',
 'coord_adj5_nome',
 'coord_adj5_tel01',
 'coord_adj5_tel01ramal',
 'coord_adj5_tel02',
 'coord_adj5_tel02r

## Salva


In [26]:
# Results
df.to_csv(
    output_path_tabs / 'tab_defesacivil.csv',
    index=False,
)
df

,repdec_cod,repdec_nome,coord_regional_mail01,coord_regional_mail02,coord_regional_nome,coord_regional_tel01,coord_regional_tel01ramal,coord_regional_tel02,coord_regional_tel02ramal,coord_adj1_mail01,...,coord_adj7_tel01ramal,coord_adj7_tel02,coord_adj7_tel02ramal,coord_adj8_mail01,coord_adj8_mail02,coord_adj8_nome,coord_adj8_tel01,coord_adj8_tel01ramal,coord_adj8_tel02,coord_adj8_tel02ramal
0,M-2,Região Metropolitana de Santo André,alexandremota@policiamilitar.sp.gov.br,None,Capitão PM Alexandre Alves Mota,(11) 4125-9108,None,None,None,nelioperantoni@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
1,M-4,Região Metropolitana de Osasco,tbelucci@policiamilitar.sp.gov.br,None,Capitão PM Thiago Oliveira Belucci,(11) 4199-2955,None,None,None,dboros@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
2,I-1,Região de Registro,kathleen.chaves6@gmail.com,None,Kathleen Gomes da Silva Chaves,(13) 3821-3244,None,None,None,celio.de.farias@hotmail.com,...,None,None,None,,None,,,None,None,None
3,I-8,Região de São José do Rio Preto,aobvioslo@policiamilitar.sp.gov.br,None,Capitão PM Alexandre Obvioslo Neto,(17) 3211-8870,None,None,None,carbonera@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
4,I-7,Região de Bauru,helderkato@policiamilitar.sp.gov.br,None,Tenente Coronel PM Helder Hideaki Kato,(14) 3222-7474,None,None,None,tozi@policiamilitar.sp.gov.br,...,None,None,None,marcelodias@bauru.sp.gov.br,None,Marcelo Ryal Dias,(14) 3235-1169,None,None,None
5,I-3,Região de São José dos Campos,pauloreis@policiamilitar.sp.gov.br,None,Major PM Paulo Roberto Reis Teixeira de Souza,(12) 3883-1717,None,(12) 3133-1438,None,ericamacedo@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
6,I-9,Região de Araçatuba,peron@policiamilitar.sp.gov.br,None,Major PM Wagner Francisco Peron,(18) 3623-3355,None,None,None,spessotto@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
7,M-3,Região Metropolitana de Guarulhos,carlosbernardes@policiamilitar.sp.gov.br,None,Major PM Antônio Carlos Bernardes,(11) 4799-1234,None,None,None,diegotonon@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
8,M-1,Região Metropolitana de São Paulo,apisani@sp.gov.br,None,Alfredo Pisani,(11) 3293-8585,None,None,None,cesartadeu@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
9,I-14,Região de Franca,sandraandrade@policiamilitar.sp.gov.br,None,Capitão PM Sandra Elaine de Andrade Bueno de C...,(16) 3712-9200,None,None,None,gcustodio@policiamilitar.sp.gov.br,...,None,None,None,,None,,,None,None,None
